In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

import findspark
findspark.init()
findspark.find()

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark = SparkSession \
       .builder \
       .appName("Our First Spark example") \
       .getOrCreate()

spark

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [806 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,517 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://ppa.launchpad.net/deadsnakes/ppa/ubunt

In [2]:
from pyspark import SparkFiles

In [3]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-06-14 07:07:25--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.50MB/s    in 0.2s    

2022-06-14 07:07:26 (5.50 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [5]:
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Sports_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
sports_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Sports_v1_00.tsv.gz"), sep="\t", header=True)
sports_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48945260|R1WBPB8MDCCN8F|B012P7UPSM|     409940130|Chicago Blackhawk...|          Sports|          5|            0|          0|   N|                N|   LOVE IT. 6 stars!|Bought this last ...| 2015-08-31|
|         US|    5782091|R32M0YEWV77XG8|B001GQ3VHG|     657746679|Copag Poker Size ...|          Sports|          5|    

In [6]:
sports_df.count()

4850360

In [7]:
from pyspark.sql.functions import to_date
# Review DataFrame
review_id_df = sports_df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1WBPB8MDCCN8F|   48945260|B012P7UPSM|     409940130| 2015-08-31|
|R32M0YEWV77XG8|    5782091|B001GQ3VHG|     657746679| 2015-08-31|
| RR8V7WR27NXJ5|   45813853|B008VS8M58|     962442336| 2015-08-31|
|R1MHO5V9Z932AY|    1593730|B005F06F4U|      74305227| 2015-08-31|
|R16PD71086BD2V|   29605511|B010T4IE2C|     787185588| 2015-08-31|
|R1Z8IFGWTRWXT6|   11112959|B004RKJGLS|      94127483| 2015-08-31|
|R3AUMSHAW73HWN|     108031|B005V3DCBU|     526977496| 2015-08-31|
|R2KWDWFOHGX6FL|   13981540|B00MHT9WN8|      26521265| 2015-08-31|
|R3H9543FWBWFBU|   37993909|B001CSIRQ8|     652431165| 2015-08-31|
| RUANXOQ9W3OU5|   26040213|B001KZ3NOO|     635861713| 2015-08-31|
|R31673RTGEZSW7|   34657602|B00005RCQS|      72099763| 2015-08-31|
|R22OQLFSH42RCM|   14346192|B00FA7RWVI|     757354022| 2015-08

In [8]:
products_df = sports_df.select(["product_id", "product_title"]).drop_duplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B0092702VE|Abu Garcia Revo S...|
|B00HZ9WTVC|Lee 90355 Cutter ...|
|B00WS4FRQ8|BEEWARE Targets -...|
|B00AKJVYYC|NEW Callaway RAZR...|
|B00DIPSOM0|GoSports Ladder T...|
|B0017LEUCU|Body Sport Exerci...|
|B00B59OMS6|Swiftwick SUSTAIN...|
|B00KC5SYLU|EliteTek RG-14 Fo...|
|B000GKUAB8|Multisport Helmet...|
|B001P80Z10|Izzo Golf Mini Mo...|
|B0063HVE18|Weston 30-0008-W ...|
|B00UR4HHDM|  Mission Multi-Cool|
|B000P3WQ86|Plano Pro-Max Sin...|
|B00005UDHY|Speedo Hydrospex ...|
|B00NZCXMUA|Fastfender Sail F...|
|B000YPIENQ|5.11 Tactical #73...|
|B00I3P5PM2|Tfo Gts Inshore G...|
|B009SPBB10|Twins Special Ful...|
|B00WIWYBZI|Black Anime Foam ...|
|B004SRA7GI|Moultrie Game Spy...|
+----------+--------------------+
only showing top 20 rows



In [9]:
customers_df = sports_df.groupby("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   45193257|            21|
|   44934924|             1|
|   11614909|             1|
|   19186664|             1|
|   50492011|             1|
|   12988293|             3|
|   22848554|             4|
|   42126527|             1|
|   48782599|             1|
|   34146651|            10|
|   43194960|             3|
|   14127895|             1|
|   30636778|            44|
|   30274993|             1|
|   28069251|             1|
|   20545025|             5|
|   28029004|             4|
|   51290010|             1|
|   37367363|             1|
|    2583392|             7|
+-----------+--------------+
only showing top 20 rows



In [10]:
vine_df = sports_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1WBPB8MDCCN8F|          5|            0|          0|   N|
|R32M0YEWV77XG8|          5|            1|          1|   N|
| RR8V7WR27NXJ5|          1|            0|          0|   N|
|R1MHO5V9Z932AY|          5|            0|          0|   N|
|R16PD71086BD2V|          5|            0|          1|   N|
|R1Z8IFGWTRWXT6|          3|            0|          0|   N|
|R3AUMSHAW73HWN|          4|            2|          3|   N|
|R2KWDWFOHGX6FL|          5|            0|          0|   N|
|R3H9543FWBWFBU|          5|            1|          1|   N|
| RUANXOQ9W3OU5|          5|            0|          0|   N|
|R31673RTGEZSW7|          5|            2|          2|   N|
|R22OQLFSH42RCM|          5|            1|          1|   N|
|R12LEL4F3TSZUJ|          5|            2|          2|   N|
|R2L9XWD03072NI|          5|            

In [11]:
mode = "append"
jdbc_url="jdbc:postgresql://mypostgresdb.ctmibqgleovv.us-east-1.rds.amazonaws.com:5432/my_data_class_db"
config = {"user":"root", "password": "Myroot1!", "driver":"org.postgresql.Driver"}

In [12]:
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

Py4JJavaError: ignored

In [ ]:
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
vine_df.write.jdbc(url=jdbc_url, table='vines', mode=mode, properties=config)